In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
from yahooquery import Ticker
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Lista de tickers
tickers = ["TTWO", "TCEHY", "EA", "RBLX", "NCBDF"]

# Funções de coleta e retorno

def collect_data(tickers, start="2019-01-01"):
    tq = Ticker(tickers)
    df = tq.history(start=start).reset_index()
    prices = df.pivot(index="date", columns="symbol", values="adjclose").dropna()
    return prices

prices = collect_data(tickers)
log_returns = np.log(prices / prices.shift(1)).dropna()
log_returns.head()

symbol,EA,NCBDF,RBLX,TCEHY,TTWO
date,,,,,
2021-03-11,0.009332,0.0,0.061386,0.053961,0.034526
2021-03-12,-0.006492,0.0,-0.058513,-0.078131,-0.002112
2021-03-15,0.003060,0.0,0.034547,0.000605,-0.006364
2021-03-16,0.006473,0.0,0.065058,-0.011810,0.022445
2021-03-17,-0.008691,0.0,-0.002731,-0.006759,-0.023983


In [ ]:
# Volume e open interest (zero se indisponível)
volume = Ticker(tickers).history(start="2019-01-01").reset_index().pivot(
    index="date", columns="symbol", values="volume").dropna()
# Ignora openInterest se não existir

# Função de montagem
def build_matrix(lr, vol):
    mean_ret = lr.mean()
    std_ret = lr.std()
    liq = vol.mean()
    market = lr.mean(axis=1)
    corr = lr.corrwith(market)
    return pd.DataFrame({
        'retorno': mean_ret,
        'risco': std_ret,
        'liquidez': liq,
        'correlacao': corr
    })

decision_df = build_matrix(log_returns, volume)
decision_df

,retorno,risco,liquidez,correlacao
symbol,,,,
EA,0.000134,0.015445,2.328463e+06,0.451627
NCBDF,0.000326,0.050652,2.308582e+02,0.540867
RBLX,-0.000054,0.045800,1.331079e+07,0.690640
TCEHY,-0.000239,0.028423,3.739077e+06,0.524324
TTWO,0.000290,0.020596,1.752475e+06,0.550317


In [ ]:
def normalize_df(df):
    nd = df.copy()
    for c in df.columns:
        if c in ['risco','correlacao']:
            nd[c] = df[c].min() / df[c]
        else:
            nd[c] = df[c] / df[c].max()
    return nd

R = normalize_df(decision_df)
R

,retorno,risco,liquidez,correlacao
symbol,,,,
EA,0.410642,1.000000,0.174930,1.000000
NCBDF,1.000000,0.304920,0.000017,0.835004
RBLX,-0.166989,0.337224,1.000000,0.653925
TCEHY,-0.734616,0.543390,0.280906,0.861351
TTWO,0.891301,0.749879,0.131658,0.820666


In [ ]:
weights = {
    'baixa':        np.array([0.10,0.35,0.45]),
    'estabilidade': np.array([0.02,0.10,0.12]),
    'alta':         np.array([0.04,0.10,0.22])
}

def fuzzy_topsis(R, weights):
    results = {}
    for cond, w_ret in weights.items():
        w_risk = 1 - w_ret
        cc = pd.DataFrame(index=R.index, columns=['inferior','modal','superior'])
        for lvl,(wr,wk) in zip(cc.columns, zip(w_ret, w_risk)):
            M = R[['retorno','risco']].values * np.array([wr,wk])
            pos, neg = M.max(0), M.min(0)
            d_pos = np.linalg.norm(M-pos, axis=1)
            d_neg = np.linalg.norm(M-neg,axis=1)
            cc[lvl] = d_neg/(d_pos+d_neg)
        results[cond] = cc
    return results

results = fuzzy_topsis(R, weights)
results['alta']

,inferior,modal,superior
symbol,,,
EA,0.965954,0.915189,0.821776
NCBDF,0.094188,0.217089,0.413103
RBLX,0.056822,0.094962,0.180804
TCEHY,0.340300,0.324852,0.262722
TTWO,0.642753,0.657274,0.717171


In [ ]:
probs = {'baixa':0.3,'estabilidade':0.5,'alta':0.2}
exp_cc = pd.Series(0, index=results['baixa'].index)
for cond,p in probs.items():
    exp_cc += results[cond]['modal'] * p
exp_cc = exp_cc.sort_values(ascending=False)
exp_cc

symbol
EA       0.864258
TTWO     0.697891
NCBDF    0.323963
TCEHY    0.283369
RBLX     0.142010
dtype: float64

In [ ]:
def interpret_cc(results):
    """Imprime top3, bottom3, robustos, sensíveis e líderes consistentes, e gera gráficos em Plotly."""
    for cond, df in results.items():
        # Mapeia coluna 'symbol' para 'ativo' para Plotly
        d = df.reset_index().rename(columns={'symbol':'ativo'})
        # Gráfico de barras com barras de erro fuzzy
        fig = px.bar(
            d,
            x='ativo',
            y='modal',
            error_y=d['superior'] - d['modal'],
            error_y_minus=d['modal'] - d['inferior'],
            title=f"Condição {cond.capitalize()} — CC_modal com barras fuzzy",
            labels={"modal":"CC_modal","ativo":"Ativo"}
        )
        fig.update_layout(xaxis_tickangle=-45)
        fig.show()

# Chamamos a função para cada cenário
evaluate_interpretation = interpret_cc(results)

✅ Proposta de novos critérios e seus motivos:
1. Sharpe Ratio (retorno ajustado ao risco)
Motivo: combina retorno e risco, permitindo comparar investimentos com diferentes níveis de volatilidade.

Fórmula: (Retorno médio - Taxa livre de risco) / Volatilidade

Pode assumir taxa livre de risco como 0 para simplificação.

2. Drawdown máximo
Motivo: mostra a pior queda acumulada do ativo, importante para avaliação de risco extremo.

Ajuda investidores mais avessos a perdas.

3. Retorno anualizado
Motivo: normaliza o retorno no tempo, facilitando comparação com benchmarks e entre ativos.

4. Beta
Motivo: mede a sensibilidade da ação em relação ao mercado.

Ações com beta > 1 são mais voláteis que o mercado; < 1 são menos voláteis.

5. Skewness (Assimetria)
Motivo: indica se os retornos têm mais chance de gerar perdas extremas ou ganhos extremos.

Ajuda a identificar ativos com retornos não simétricos.

6. Volatilidade histórica anualizada
Motivo: melhora a interpretação do risco comparando em termos anuais.

✅ Outras ideias

1. Sortino Ratio
Variação do Sharpe, mas penaliza apenas a volatilidade negativa (quedas).

Mais justo para ativos com volatilidade positiva (bons retornos).

2. Alfa (Alpha)
Mede o retorno excedente em relação ao mercado, ajustado ao risco (Beta).

Se alfa > 0 → o ativo superou o mercado.

3. Índice de Treynor
Parecido com Sharpe, mas ajusta pelo Beta ao invés da volatilidade.

Bom para medir desempenho com base no risco sistemático.

4. Desvio Padrão das Quedas (Downside Deviation)
Útil para calcular o Sortino Ratio.

Foca apenas nas perdas — muito usado por investidores conservadores.

5. Valor em Risco (VaR) - Paramétrico ou Histórico
Mede a perda máxima esperada com um certo nível de confiança.

Exemplo: “Com 95% de confiança, você não perderá mais que X% em um dia.”

📈 Ideias de visualizações para complementar:
Gráfico de bolhas: eixos com risco e retorno, tamanho da bolha representando liquidez.

Mapa de calor de correlação: entre os ativos.

Ranking ponderado: combinando critérios com pesos atribuídos por você.

Radar Chart (Gráfico de aranha): para comparar todos os critérios entre ativos.

Clusterização K-means: agrupar ativos com perfil similar (ex: agressivos, moderados, conservadores).

⚙️ Melhorias na matriz de decisão:
Normalização dos critérios (0–1) para facilitar comparação e ranking.

Criação de um Final Score, combinando os critérios com pesos (como um "Índice de Qualidade").

Adição de metas ou filtros: ex., remover ativos com liquidez muito baixa ou drawdown extremo.